## Setup

Downloads the FHIR Validator from https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar

For more details on the FHIR Validator see [Using the FHIR Validator](https://confluence.hl7.org/spaces/FHIR/pages/35718580/Using+the+FHIR+Validator)

Downloads the NW Genomics validation package

In [1]:
from tqdm import tqdm
import requests

print("Downloading the FHIR Validator...")

url = "https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar"
response = requests.get(url, stream=True)

with open("validator_cli.jar", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)

print("Downloading the North West Genomics validation package...")
url = "https://nw-gmsa.github.io/package.tgz"
response = requests.get(url, stream=True)

with open("package.tgz", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)


181476kB [00:05, 32053.80kB/s]


4603kB [00:00, 7133.06kB/s]


Now run a validation using a FHIR Message Bundle.

The example is an O21 Order, so we also need to help the validator by specifying the profile of the focus resource which in this case is a ServiceRequest which should adhere to https://nw-gmsa.github.io/StructureDefinition-ServiceRequest.html (profile url https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest)

A default output is returned from the validator.

Using command line this is:

>  java -jar validator_cli.jar Input/FHIR/O21/Liverpool_O21.json -version 4.0.1 -ig package.tgz -bundle ServiceRequest:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest -tx n/a

In [2]:
import subprocess

filename = "Liverpool_O21.json"
folder = 'FHIR/R01/'

subprocess.run(["java", "-jar", "validator_cli.jar", "Input/"+folder + filename , "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "ServiceRequest:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest", "-tx", "n/a"])

FHIR Validation tool Version 6.8.0 (Git# 16897818c932). Built 2026-02-03T21:15:40.739Z (12 days old)
  Java:   21.0.7 from /Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home on x86_64 (64bit). 8192MB available
  Paths:  Current = /Users/kevinmayfield/github/MFT/Testing, Package Cache = /Users/kevinmayfield/.fhir/packages
  Params: Input/FHIR/O21/Liverpool_O21.json -version 4.0.1 -ig package.tgz -bundle ServiceRequest:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest -tx n/a
  Locale: United Kingdom/GB
  Jurisdiction: United Kingdom of Great Britain and Northern Ireland
Loading
  Loading FHIR v4.0.1 from hl7.fhir.r4.core#4.0.1
  Load hl7.terminology.r4#6.2.0 - 4288 resources (00:13.359)
  Load hl7.fhir.uv.extensions.r4#5.2.0 - 759 resources (00:03.358)
  Loaded FHIR - 8265 resources (00:00.000)
  Terminology server null - Version n/a: No Terminology Server (00:00.000)
  Load hl7.fhir.uv.extensions.r5#5.2.0 - 759 resources (00:01.414)
  Load hl7.terminology#7.0.

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Input/FHIR/O21/Liverpool_O21.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'ServiceRequest:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest', '-tx', 'n/a'], returncode=0)

The output can contain a mix of issues which may be difficult to understand. If we get a large number of issues, we may want to prioritise `error` first and write a test report. So we need to output the results in a machine readable format



In [3]:
import subprocess

subprocess.run(["java", "-jar", "validator_cli.jar", "Input/"+folder + filename, "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "ServiceRequest:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest", "-tx", "n/a", "-output", "Results/"+folder + filename + "-OperationOutcome.json", "-output-style","json"])

FHIR Validation tool Version 6.8.0 (Git# 16897818c932). Built 2026-02-03T21:15:40.739Z (12 days old)
  Java:   21.0.7 from /Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home on x86_64 (64bit). 8192MB available
  Paths:  Current = /Users/kevinmayfield/github/MFT/Testing, Package Cache = /Users/kevinmayfield/.fhir/packages
  Params: Input/FHIR/O21/Liverpool_O21.json -version 4.0.1 -ig package.tgz -bundle ServiceRequest:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest -tx n/a -output Results/FHIR/O21/Liverpool_O21.json-OperationOutcome.json -output-style json
  Locale: United Kingdom/GB
  Jurisdiction: United Kingdom of Great Britain and Northern Ireland
Loading
  Loading FHIR v4.0.1 from hl7.fhir.r4.core#4.0.1
  Load hl7.terminology.r4#6.2.0 - 4288 resources (00:10.658)
  Load hl7.fhir.uv.extensions.r4#5.2.0 - 759 resources (00:02.779)
  Loaded FHIR - 8265 resources (00:00.000)
  Terminology server null - Version n/a: No Terminology Server (00:00.000)
  Load hl

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Input/FHIR/O21/Liverpool_O21.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'ServiceRequest:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest', '-tx', 'n/a', '-output', 'Results/FHIR/O21/Liverpool_O21.json-OperationOutcome.json', '-output-style', 'json'], returncode=0)

In [4]:
import json
import pandas as pd

def details(item):
    return item['text']

with open( "Results/"+folder + filename + "-OperationOutcome.json", 'rb') as f:
    s = f.read().decode('utf-8')
    operationOutcome = json.loads(s)

df = pd.DataFrame(operationOutcome['issue'])

df['details'] = df['details'].apply(details)
df.drop(columns=['extension'], inplace=True)
df.sort_values(by=['severity'], inplace=True)


df = df[~df['details'].str.contains('ValueSet/mimetypes')]
df = df[~df['details'].str.contains('failed: dom-6')]
df = df[~df['details'].str.contains('bcp:13')]

df

,severity,code,details,expression
135,error,structure,"ServiceRequest.status: minimum required = 1, b...",[Bundle.entry[1].resource]
1,error,structure,MessageHeader.source.endpoint: minimum require...,[Bundle.entry[0].resource/*MessageHeader/64001...
3,error,structure,"ServiceRequest.status: minimum required = 1, b...",[Bundle.entry[1].resource/*ServiceRequest/ee6a...
139,error,structure,Unable to find a profile match for urn:uuid:dc...,[Bundle.entry[1].resource.subject]
110,error,structure,"Observation.status: minimum required = 1, but ...",[Bundle.entry[20].resource/*Observation/735aae...
...,...,...,...,...
55,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[11].resource/*Observation/OBX-20...
54,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[11].resource/*Observation/OBX-20...
50,warning,invalid,Coding has no system. A code with no system ha...,[Bundle.entry[10].resource/*Observation/OBX-8-...
49,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[10].resource/*Observation/OBX-8-...


In [5]:
import jinja2
import weasyprint
from datetime import date

html_string = df.to_html()

html = (
    jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath=""))
    .get_template(name="Results/report_template.html")
    .render(
        date=date.today().strftime("%d, %b %Y"),
        table=html_string
    )
)

with open("Results/"+folder + filename + "-report.html", "w") as out_html_file_handle:
    out_html_file_handle.write(html)

with open("Results/"+folder + filename + "-report.html", "r") as html_file_handle:
    pdf = weasyprint.HTML(html_file_handle).write_pdf()


open("Results/"+folder + filename + '-report.pdf', 'wb').write(pdf)



31203